In [ ]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 84.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import geopandas as gpd
import folium
from shapely.geometry import Point
import matplotlib.pyplot as plt 
from IPython.display import IFrame
from google.colab import files


#identify what we are predicting
#which class
#regr prob, classi prob, clustering prob
#use scikitlearn/sklearn

In [ ]:

# Set the file path 
# Both files work on the same code just change the path for the file you trying to use
file_path = '/content/riofile1 (2017_10_09 12_59_46 UTC).json'

# Read the JSON file into a DataFrame
# use a json parser
df = pd.read_json(file_path, lines=True)

# Print the first 5 rows of the DataFrame
df.head()

,created_at,user,lang,retweeted,possibly_sensitive,place,metadata,geo,in_reply_to_user_id_str,is_quote_status,...,entities,id,id_str,truncated,retweet_count,coordinates,in_reply_to_status_id,quoted_status_id,quoted_status_id_str,quoted_status
0,2015-10-26 21:37:57+00:00,"{'profile_use_background_image': True, 'profil...",pt,False,0.0,None,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,False,...,"{'symbols': [], 'urls': [{'url': 'https://t.co...",658759495162712064,658759495162712064,False,5,None,NaN,NaN,NaN,NaN
1,2015-10-26 21:37:03+00:00,{'profile_image_url': 'http://pbs.twimg.com/pr...,pt,False,0.0,"{'place_type': 'city', 'attributes': {}, 'id':...","{'iso_language_code': 'pt', 'result_type': 're...","{'coordinates': [-22.80625062, -43.19914341], ...",NaN,False,...,"{'user_mentions': [{'id': 1897952138, 'id_str'...",658759268007608320,658759268007608320,False,0,"{'coordinates': [-43.19914341, -22.80625062], ...",NaN,NaN,NaN,NaN
2,2015-10-26 21:37:47+00:00,"{'contributors_enabled': False, 'time_zone': '...",pt,False,0.0,{'bounding_box': {'coordinates': [[[-43.940006...,"{'iso_language_code': 'pt', 'result_type': 're...","{'type': 'Point', 'coordinates': [-22.8670353,...",NaN,False,...,"{'hashtags': [{'text': 'AniversáriodaBia', 'in...",658759453999804416,658759453999804416,False,0,"{'type': 'Point', 'coordinates': [-43.74816589...",NaN,NaN,NaN,NaN
3,2015-10-26 21:38:03+00:00,"{'default_profile': False, 'screen_name': 'sxm...",und,False,NaN,"{'place_type': 'city', 'attributes': {}, 'full...","{'iso_language_code': 'und', 'result_type': 'r...","{'coordinates': [-22.8069661, -43.1912629], 't...",2.768488e+09,False,...,"{'urls': [], 'user_mentions': [{'screen_name':...",658759521066729472,658759521066729472,False,0,"{'coordinates': [-43.1912629, -22.8069661], 't...",6.587589e+17,NaN,NaN,NaN
4,2015-10-26 21:34:42+00:00,"{'profile_use_background_image': True, 'id': 2...",pt,False,NaN,"{'country': 'Brasil', 'url': 'https://api.twit...","{'iso_language_code': 'pt', 'result_type': 're...","{'coordinates': [-22.8250204, -42.9991559], 't...",NaN,False,...,"{'symbols': [], 'hashtags': [], 'urls': [], 'u...",658758677051121664,658758677051121664,False,0,"{'coordinates': [-42.9991559, -22.8250204], 't...",NaN,NaN,NaN,NaN


In [ ]:
# Remove irrelevant columns
location = df[['coordinates', 'id']]  
location.dropna

<bound method DataFrame.dropna of                                              coordinates                  id
0                                                   None  658759495162712064
1      {'coordinates': [-43.19914341, -22.80625062], ...  658759268007608320
2      {'type': 'Point', 'coordinates': [-43.74816589...  658759453999804416
3      {'coordinates': [-43.1912629, -22.8069661], 't...  658759521066729472
4      {'coordinates': [-42.9991559, -22.8250204], 't...  658758677051121664
...                                                  ...                 ...
31995  {'coordinates': [-43.40184053, -22.83547327], ...  658115571494617088
31996                                               None  658350449800695808
31997  {'coordinates': [-43.58133158, -22.91749511], ...  658101449876971520
31998                                               None  658101442222403584
31999  {'coordinates': [-43.4097718, -22.7434448], 't...  658303857295011840

[32000 rows x 2 columns]>

In [ ]:
initalLocationArray = location.to_numpy()
objArray = []
idDictionary = {}

counter = 0

for obj in initalLocationArray:
  objArray.append(obj)

for obj in objArray:
  if obj[1] not in idDictionary.keys() and obj[0] != None:
    idDictionary[obj[1]] = obj[0]

popList = []

for id in idDictionary.keys():
  cord = idDictionary.get(id).get('coordinates')
  if cord[0] < -43.795583 or cord[0] > -43.147969:
    popList.append(id)
  elif cord[1] < -23.067211 or cord[1] > -22.778976:
    popList.append(id)

for id in popList:
  idDictionary.pop(id)


result = idDictionary.items()
data = list(result)
numpyArray = np.array(data)

newDf = pd.DataFrame(numpyArray, columns = ['id','coordinates'])
newDf.head

newDf[['longitude', 'latitude']] = pd.DataFrame(newDf['coordinates'].tolist(), index=newDf.index)['coordinates'].apply(pd.Series)
newX = newDf[['longitude', 'latitude']].values
new_clusters = 34
newKmeans = KMeans(n_clusters = new_clusters, init='k-means++', random_state=42)
newKmeans.fit(newX)
newDf['cluster'] = newKmeans.labels_

map = folium.Map(location=[-14.235004,-51.92528], zoom_start=4)\

# define the colors for the clusters
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'lightgreen', 'gray', 'black']
# group the data points by their cluster labels
groups = newDf.groupby('cluster')

for name, group in groups:
    for i in range(len(group)):
        folium.CircleMarker(location=[group.iloc[i]['latitude'], group.iloc[i]['longitude']],
                            radius=4, color=colors[name], fill_color=colors[name], fill_opacity=0.7).add_to(map)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
map.save('clustering_map.html')
files.download('clustering_map.html')

KeyboardInterrupt: ignored

In [ ]:
# Drop any rows with missing or null values
location = location.dropna()


In [ ]:
# extract the two numbers and create new columns
location[['longitude', 'latitude']] = pd.DataFrame(location['coordinates'].tolist(), index=location.index)['coordinates'].apply(pd.Series)


In [ ]:
# drop the original 'coordinates' column if not needed
location.drop('coordinates', axis=1, inplace=True)


In [ ]:
location

In [ ]:
# extract the features for clustering
X = location[['longitude', 'latitude']].values

In [ ]:
# instantiate the KMeans object and fit the data
# don't change the n_clusters if to you did not add to the colors variable below
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
kmeans.fit(X)

In [ ]:
# add the cluster labels to the DataFrame
location['cluster'] = kmeans.labels_

In [ ]:
location

In [ ]:
# create a Folium map centered on Brazil
m = folium.Map(location=[-14.235004,-51.92528], zoom_start=4)

In [ ]:
# define the colors for the clusters
colors = ['red', 'blue', 'green', 'yellow', 'orange', 'pink', 'black', 'white', 'gray', 'brown', 'navy', 
          'turquoise', 'lavender', 'olive', 'maroon', 'teal', 'gold', 'silver', 'magenta', 'beige', 
          'cyan', 'indigo', 'fuchsia', 'lime', 'skyblue', 'salmon', 'tan', 'peach', 'mint', 'rust']


# group the data points by their cluster labels
groups = location.groupby('cluster')

In [ ]:
# iterate over each group and add the data points to the map with a different color
for name, group in groups:
    for i in range(len(group)):
        folium.CircleMarker(location=[group.iloc[i]['latitude'], group.iloc[i]['longitude']],
                            radius=5, color=colors[name], fill_color=colors[name], fill_opacity=0.8).add_to(m)


In [ ]:
# save the map as an HTML file
m.save('clustering_map.html')


In [ ]:
#after saving the file download and click on the file to open the map
files.download('clustering_map.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>